# Modelado de Series Temporales de Aparcamientos usando RNNs y optimización con Optuna
En este notebook se abordará un problema de predicción de series temporales utilizando datos de disponibilidad de parkings.

Flujo de trabajo:
1. Agrupación de los datos por id de aparcamiento (idAparcamiento), para tratar cada parking como una serie temporal independiente.
2. División del conjunto de datos en tres subconjuntos: entrenamiento (train), validación (val) y prueba (test).
3. Entrenamiento de modelos de redes neuronales recurrentes simples (vanilla):
 - Vanilla RNN
 - Vanilla GRU
 - Vanilla LSTM
4. Ajuste de hiperparámetros utilizando Optuna para encontrar la configuración óptima en cada tipo de modelo.
5. Comparación de los resultados de rendimiento entre los distintos modelos utilizando métricas apropiadas.

El objetivo final es evaluar qué arquitectura ofrece mejores resultados para este tipo de datos y tarea de predicción.


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn
import optuna


## 1. Cargar los datos

In [8]:
df = pd.read_csv("../data/processed/data_processed.csv")

#convertir a indice
df.set_index("timestamp", inplace= True)
df.index = pd.to_datetime(df.index)

df

,idAparcamiento,PlazasTotales,PlazasDisponibles,PorcPlazasDisponibles,year,month,day,weekday
timestamp,,,,,,,,
2023-02-03 10:00:00,6,372.0,60.0,16.129032,2023,2,3,4
2023-02-03 11:00:00,6,372.0,48.0,12.903226,2023,2,3,4
2023-02-03 12:00:00,6,372.0,66.0,17.741935,2023,2,3,4
2023-02-03 13:00:00,6,372.0,119.0,31.989247,2023,2,3,4
2023-02-03 14:00:00,6,372.0,155.0,41.666667,2023,2,3,4
...,...,...,...,...,...,...,...,...
2025-03-05 03:00:00,78,464.0,355.0,76.508621,2025,3,5,2
2025-03-05 04:00:00,78,464.0,355.0,76.508621,2025,3,5,2
2025-03-05 05:00:00,78,464.0,356.0,76.724138,2025,3,5,2


## 2. División del conjunto de datos

Realizamos la división del conjunto de datos, agrupando por `idAparcamiento`. 
La idea es que el conjunto de **test sea común** para todos los aparcamientos, correspondiente al **último 10% del rango temporal total** del dataset. 

El resto de los datos disponibles para cada parking se dividen en:

- **Entrenamiento (train)**: el primer 85% de los datos previos al test.
- **Validación (val)**: el último 15% de los datos previos al test.

In [9]:
from datetime import timedelta

# 1. Calcular el rango temporal global
fecha_min_global = df.index.min()
fecha_max_global = df.index.max()
rango_total = fecha_max_global - fecha_min_global

# 2. Calcular el inicio del conjunto de test (último 10% del rango)
test_ratio = 0.10
test_duration = timedelta(seconds=rango_total.total_seconds() * test_ratio)
test_start = fecha_max_global - test_duration

# 3. Diccionarios para almacenar los splits
train_dict = {}
val_dict = {}
test_dict = {}

val_ratio = 0.1  # del conjunto anterior al test

# 4. División por parking
for parking_id, group in df.groupby("idAparcamiento"):
    group = group.sort_index()
    
    # Split basado en el corte global
    test_set = group[group.index >= test_start]
    remaining = group[group.index < test_start]

    # Dividir en train y val
    val_size = int(len(remaining) * val_ratio)
    val_set = remaining.iloc[-val_size:]
    train_set = remaining.iloc[:-val_size]
    
    # Guardar resultados
    train_dict[parking_id] = train_set
    val_dict[parking_id] = val_set
    test_dict[parking_id] = test_set




In [10]:
train_dict[6]

,idAparcamiento,PlazasTotales,PlazasDisponibles,PorcPlazasDisponibles,year,month,day,weekday
timestamp,,,,,,,,
2023-02-03 10:00:00,6,372.0,60.00,16.129032,2023,2,3,4
2023-02-03 11:00:00,6,372.0,48.00,12.903226,2023,2,3,4
2023-02-03 12:00:00,6,372.0,66.00,17.741935,2023,2,3,4
2023-02-03 13:00:00,6,372.0,119.00,31.989247,2023,2,3,4
2023-02-03 14:00:00,6,372.0,155.00,41.666667,2023,2,3,4
...,...,...,...,...,...,...,...,...
2024-09-05 17:00:00,6,372.0,189.68,50.989247,2024,9,5,3
2024-09-05 18:00:00,6,372.0,190.28,51.150538,2024,9,5,3
2024-09-05 19:00:00,6,372.0,195.08,52.440860,2024,9,5,3


## 3. Definir `Dataset` de pytorch

In [14]:
from torch.utils.data import Dataset

In [23]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, input_window, output_window, feature_cols, target_col):
        """
        data: DataFrame que contiene los datos de la serie temporal
        input_window: nº de pasos de tiempo en la secuencia de entrada
        output_window: nº de pasos de tiempo a predecir
        feature_cols: lista de nombres de columnas que se usan como característcas
        target_col: nombre de la variable a predecir
        """
        self.data = data
        self.input_window = input_window
        self.output_window = output_window
        self.feature_cols = feature_cols
        self.target_cols = target_col

    def __len__(self):
        """
        Función que devuele el nº de datos del Dataset
        """
        return len(self.data) - self.input_window - self.output_window + 1 #

    def __getitem__(self, idx):
        """
        Función que devuelve un dato a partir de un índice
        """
        X = self.data[idx: idx + self.input_window][self.feature_cols].values
        Y = self.data[idx + self.input_window: idx + self.input_window + self.output_window][self.target_cols].values
        
        return X, Y

In [24]:
train_dict[6]

,idAparcamiento,PlazasTotales,PlazasDisponibles,PorcPlazasDisponibles,year,month,day,weekday
timestamp,,,,,,,,
2023-02-03 10:00:00,6,372.0,60.00,16.129032,2023,2,3,4
2023-02-03 11:00:00,6,372.0,48.00,12.903226,2023,2,3,4
2023-02-03 12:00:00,6,372.0,66.00,17.741935,2023,2,3,4
2023-02-03 13:00:00,6,372.0,119.00,31.989247,2023,2,3,4
2023-02-03 14:00:00,6,372.0,155.00,41.666667,2023,2,3,4
...,...,...,...,...,...,...,...,...
2024-09-05 17:00:00,6,372.0,189.68,50.989247,2024,9,5,3
2024-09-05 18:00:00,6,372.0,190.28,51.150538,2024,9,5,3
2024-09-05 19:00:00,6,372.0,195.08,52.440860,2024,9,5,3


In [28]:
feature_cols = ['PorcPlazasDisponibles']  
target_col = 'PorcPlazasDisponibles'     
input_window = 24         # Número de pasos de tiempo en la secuencia de entrada
output_window = 1         # Número de pasos de tiempo a predecir

train_dataset = TimeSeriesDataset(train_dict[6], input_window, output_window, feature_cols, target_col)
# val_dataset = TimeSeriesDataset(val_df, input_window, output_window, feature_cols, target_col)
# test_dataset = TimeSeriesDataset(test_df, input_window, output_window, feature_cols, target_col)


In [29]:
train_dataset[0]

(array([[16.12903226],
        [12.90322581],
        [17.74193548],
        [31.98924731],
        [41.66666667],
        [44.08602151],
        [46.50537634],
        [43.5483871 ],
        [41.93548387],
        [47.31182796],
        [47.04301075],
        [49.46236559],
        [56.4516129 ],
        [66.12903226],
        [71.50537634],
        [72.31182796],
        [72.58064516],
        [72.84946237],
        [72.84946237],
        [73.11827957],
        [72.84946237],
        [72.84946237],
        [69.62365591],
        [64.51612903]]),
 array([50.80645161]))

## 4. Crear `DataLoaders` a partir de `Dataset`

## 5. Definir Modelos (`RNN`,`GRU`,`LSTM`)


## 6. Definir métricas

## 7. Instanciar optimizador, función de coste y learning rate scheduler

## 8. Crear callbacks

## 9. Definir bucle de entrenamiento

## 10. Definir bucle de validación

## 11. Optimizació óptima de hiperparámetros con `Optuna`

## 12. Entrenamiento

## 13. Predicciones sobre el conjunto de test

## 14. Exportación de checkpoints y logs